In [1]:
import pandas as pd
from torch.nn.functional import softmax
import torch


pseudolab = pd.read_csv("pseudo/pseudolab.csv")
pseudolab

,sentence,Claim,Concluding Statement,Counterclaim,Evidence,Lead,Position,Rebuttal,text
0,In 2019 a wave of anti-abortion laws swept thi...,6.635360,-2.915717,-0.175175,1.890639,-2.265393,-2.164053,-0.640354,In 2019 a wave of anti-abortion laws swept thi...
1,But these grabbed the public’s attention in a ...,-0.422903,-1.966259,0.220908,0.146967,-3.457461,-0.901608,6.126251,In 2019 a wave of anti-abortion laws swept thi...
2,Georgia banned abortion after about six weeks ...,1.859675,-3.230959,-0.389729,6.427304,-1.808647,-3.455705,-0.482240,In 2019 a wave of anti-abortion laws swept thi...
3,"Ohio, Mississippi, Louisiana and Kentucky did ...",0.327022,-2.367975,-1.594195,7.579478,-1.537600,-3.283216,-0.565419,In 2019 a wave of anti-abortion laws swept thi...
4,"Alabama went the furthest, banning virtually a...",6.683917,-2.563557,-0.671641,1.514271,-2.902210,-1.887146,-0.006093,In 2019 a wave of anti-abortion laws swept thi...
...,...,...,...,...,...,...,...,...,...
28359,"Jim Justice of West Virginia, a Republican, sa...",-0.826471,-2.144290,-0.996875,8.095543,-1.530808,-3.267610,-0.607475,WASHINGTON — The Biden administration has claw...
28360,The Biden administration had hoped to avoid sh...,3.889133,-3.152361,1.242280,2.615246,-2.932186,-2.632563,0.941492,WASHINGTON — The Biden administration has claw...
28361,"Late last year, the White House persuaded Ariz...",5.278596,-2.919935,-0.322822,2.819602,-2.090031,-2.463582,-0.311356,WASHINGTON — The Biden administration has claw...
28362,Yet administration officials are less concerne...,-0.367214,-2.063666,0.582598,0.422793,-3.704168,-1.220427,6.007252,WASHINGTON — The Biden administration has claw...


In [2]:
category_codes = {0: 'Claim',
 1: 'Concluding Statement',
 2: 'Counterclaim',
 3: 'Evidence',
 4: 'Lead',
 5: 'Position',
 6: 'Rebuttal'}
labels = list(zip(*category_codes.items()))[1]
labels

('Claim',
 'Concluding Statement',
 'Counterclaim',
 'Evidence',
 'Lead',
 'Position',
 'Rebuttal')

In [3]:
def softermax(row):
    return pd.Series([float(i) for i in softmax(torch.tensor(row.tolist()), dim=-1)])

probs = pseudolab[list(labels)].apply(softermax, 1)
probs.columns = labels
probs

,Claim,Concluding Statement,Counterclaim,Evidence,Lead,Position,Rebuttal
0,0.989267,0.000070,0.001090,0.008604,0.000135,0.000149,0.000685
1,0.001420,0.000303,0.002703,0.002511,0.000068,0.000880,0.992114
2,0.010251,0.000063,0.001081,0.987308,0.000262,0.000050,0.000986
3,0.000708,0.000048,0.000104,0.998722,0.000110,0.000019,0.000290
4,0.992139,0.000096,0.000634,0.005642,0.000068,0.000188,0.001233
...,...,...,...,...,...,...,...
28359,0.000133,0.000036,0.000112,0.999475,0.000066,0.000012,0.000166
28360,0.710977,0.000622,0.050390,0.198890,0.000775,0.001046,0.037300
28361,0.913836,0.000251,0.003374,0.078152,0.000576,0.000397,0.003413
28362,0.001686,0.000309,0.004359,0.003715,0.000060,0.000718,0.989153


In [4]:
nytimes_fil = pseudolab[pseudolab.text.isin(pd.concat((pseudolab, pd.DataFrame({"sigma":probs.std(axis=1)})), axis=1)[["sentence", "text", "sigma"]].groupby("text").sigma.mean().sort_values().iloc[103:].index.tolist())].reset_index().drop(columns="index")
nytimes_fil

,sentence,Claim,Concluding Statement,Counterclaim,Evidence,Lead,Position,Rebuttal,text
0,In 2019 a wave of anti-abortion laws swept thi...,6.635360,-2.915717,-0.175175,1.890639,-2.265393,-2.164053,-0.640354,In 2019 a wave of anti-abortion laws swept thi...
1,But these grabbed the public’s attention in a ...,-0.422903,-1.966259,0.220908,0.146967,-3.457461,-0.901608,6.126251,In 2019 a wave of anti-abortion laws swept thi...
2,Georgia banned abortion after about six weeks ...,1.859675,-3.230959,-0.389729,6.427304,-1.808647,-3.455705,-0.482240,In 2019 a wave of anti-abortion laws swept thi...
3,"Ohio, Mississippi, Louisiana and Kentucky did ...",0.327022,-2.367975,-1.594195,7.579478,-1.537600,-3.283216,-0.565419,In 2019 a wave of anti-abortion laws swept thi...
4,"Alabama went the furthest, banning virtually a...",6.683917,-2.563557,-0.671641,1.514271,-2.902210,-1.887146,-0.006093,In 2019 a wave of anti-abortion laws swept thi...
...,...,...,...,...,...,...,...,...,...
23458,"Jim Justice of West Virginia, a Republican, sa...",-0.826471,-2.144290,-0.996875,8.095543,-1.530808,-3.267610,-0.607475,WASHINGTON — The Biden administration has claw...
23459,The Biden administration had hoped to avoid sh...,3.889133,-3.152361,1.242280,2.615246,-2.932186,-2.632563,0.941492,WASHINGTON — The Biden administration has claw...
23460,"Late last year, the White House persuaded Ariz...",5.278596,-2.919935,-0.322822,2.819602,-2.090031,-2.463582,-0.311356,WASHINGTON — The Biden administration has claw...
23461,Yet administration officials are less concerne...,-0.367214,-2.063666,0.582598,0.422793,-3.704168,-1.220427,6.007252,WASHINGTON — The Biden administration has claw...
